# AI e Aprendizado de Máquina no Databricks

##### Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro

Link de estudo:

* [docs databricks](https://docs.databricks.com/aws/pt/getting-started/quick-start)

## Consultar e visualizar dados no Databricks

In [0]:
# Só executar quando estiver trabalhando localmente:
import os
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

databricks_host = os.environ['DATABRICKS_HOST']
databricks_token = os.environ['DATABRICKS_TOKEN']
databricks_cluster_id = os.environ['DATABRICKS_CLUSTER_ID']

print("🔗 Carregado minhas credenciais do Databricks, com sucesso!")

In [0]:
# Só executar quando estiver trabalhando localmente:
from databricks.connect import DatabricksSession # Esta classe é usada para conectar ao Databricks remotamente

spark = DatabricksSession.builder.remote(host=databricks_host, token=databricks_token, cluster_id=databricks_cluster_id).getOrCreate()


print("✅ Conectado ao Databricks com sucesso!")
print(f"🔧 Versão Spark: {spark.version}")

In [0]:
databricks_cluster_id = spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

print(f"Databricks Cluster ID: {databricks_cluster_id}")

In [0]:
# Consultando a seguinte Tabela "trips" que se encontra no 'samples' do Databricks:
taxi_df = spark.read.table("samples.nyctaxi.trips")

display(taxi_df) # Pode usar, também ---> display(taxi_df.limit(10))

Databricks visualization. Run in Databricks to view.

In [0]:
# Exibir informações básicas da tabela
print("🚕 === DADOS DA TABELA NYC TAXI TRIPS ===")
print(f"📊 Total de registros (quantidade de dados/linhas): {taxi_df.count():,}")
print(f"📋 Número de colunas: {len(taxi_df.columns)}")
print(f"🏷️ O nomes das colunas são: {', '.join(taxi_df.columns)}")

print("\n📋 ESTRUTURA DA TABELA:")
taxi_df.printSchema()


In [0]:
# Usando o método show() para exibir: 
print("🔍 === PRIMEIRAS 20 LINHAS DA TABELA ===")
taxi_df.show(10, truncate=False) # Melhor usar "False" porque não vai curtar os textos longos.


A seguir vou consultar meus dados `CSV` que tenho no meu `volume` dentro do `Catalog`:

In [0]:
df_volume = spark.read.csv("/Volumes/workspace/default_eddy/volumeeddy-tmp-sampledata/sample_data.csv", header=True, inferSchema=True)

df_volume.display()

Databricks visualization. Run in Databricks to view.

## Importar e visualizar dados do CSV

Primeiro definimos, criamos o necessário para poder salvar nossos dados `"nome de bebês"` em nosso volume.

In [0]:
catalog = "catalogeddy"
schema = "catalogeddy_schema"
volume = "catalogeddy_volume"
download_url = "https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv" # São dados de nome de bebês
file_name = "baby_eddy.csv"
table_name = "tabel_baby_eddy"
path_volume = "/Volumes/" + catalog + "/" + schema + "/" + volume
path_table = catalog + "." + schema

print(f"O caminho completo da tabela é: {path_table}") 
print(f"O caminho completo do volume é:{path_volume}") 

In [0]:
# Este comando copia os dados baixados e salva dentro do volume criado anteriormente:
dbutils.fs.cp(f"{download_url}", f"{path_volume}" + "/" + f"{file_name}")

Agora que temos nossos dados salvos no volume `catalogeddy_volume` vamos a carregar esses dados `CSV` em um `DataFrame`:

In [0]:
df = spark.read.csv(f"{path_volume}/{file_name}",
  header=True,
  inferSchema=True,
  sep=",")

df.display()

Databricks visualization. Run in Databricks to view.

In [0]:
display(df)

Databricks visualization. Run in Databricks to view.

Agora, vamos salvar o `DataFrame` em uma tabela.

Na seguinte célula vamos substituir um espaço no nome da coluna. [Caracteres especiais](https://docs.databricks.com/aws/pt/sql/language-manual/sql-ref-names), como espaços, não são permitidos nos nomes das colunas.

In [0]:
df = df.withColumnRenamed("First Name", "First_Name")

df = df.withColumnRenamed("First_Name", "Primeiro_Nome") \
       .withColumnRenamed("Year", "Ano") \
           .withColumnRenamed("County", "Bairro") \
       .withColumnRenamed("Sex", "Sexo") \
       .withColumnRenamed("Count", "Contagem")

df.columns

In [0]:
df.limit(3).display()

In [0]:
# Salvando o DataFrame como Tabela:
df.write.mode("overwrite").saveAsTable(f"{path_table}" + "." + f"{table_name}")